In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd

In [2]:
import os
import skimage.io as sk

In [3]:
from torchvision.transforms import transforms

In [4]:
from torch.utils.data import DataLoader

In [5]:
from skimage.transform import resize

In [6]:
Train_Data = pd.read_csv("./CMATERdbTrain.csv", usecols=["labels","directory"])

In [7]:
Train_Data

,labels,directory
0,1,dataset\CMATERdbTrain\1\bcc000000.bmp
1,1,dataset\CMATERdbTrain\1\bcc000032.bmp
2,1,dataset\CMATERdbTrain\1\bcc000033.bmp
3,1,dataset\CMATERdbTrain\1\bcc000034.bmp
4,1,dataset\CMATERdbTrain\1\bcc000035.bmp
...,...,...
1572,156,dataset\CMATERdbTrain\156\bcc001573.bmp
1573,156,dataset\CMATERdbTrain\156\bcc001574.bmp
1574,156,dataset\CMATERdbTrain\156\bcc001572.bmp
1575,158,dataset\CMATERdbTrain\158\bcc001575.bmp


In [8]:
Test_Data = pd.read_csv("./CMATERdbTest.csv", usecols=["labels","directory"])

In [9]:
Test_Data

,labels,directory
0,1,dataset\CMATERdbTest\1\bcc000000.bmp
1,1,dataset\CMATERdbTest\1\bcc000014.bmp
2,1,dataset\CMATERdbTest\1\bcc000012.bmp
3,1,dataset\CMATERdbTest\1\bcc000011.bmp
4,1,dataset\CMATERdbTest\1\bcc000010.bmp
...,...,...
467,155,dataset\CMATERdbTest\155\bcc000467.bmp
468,156,dataset\CMATERdbTest\156\bcc000468.bmp
469,158,dataset\CMATERdbTest\158\bcc000469.bmp
470,159,dataset\CMATERdbTest\159\bcc000470.bmp


In [10]:
nuTest = Test_Data.to_numpy()
print(len(nuTest))

472


In [11]:
Test_Data.to_numpy()[0][1]

'dataset\\CMATERdbTest\\1\\bcc000000.bmp'

In [12]:
image = sk.imread(Test_Data.to_numpy()[0][1])

In [13]:
image.shape

(149, 109, 3)

In [14]:
t = transforms.ToTensor()

In [15]:
t_image = t(image)
t_image.shape

torch.Size([3, 149, 109])

In [16]:
torch_image = torch.from_numpy(image)

In [17]:
torch_image.shape

torch.Size([149, 109, 3])

In [18]:
nexts = torch_image.permute(2,0,1)

In [19]:
nexts.shape

torch.Size([3, 149, 109])

In [20]:
class CompundCharacterDataset(Dataset):
    def __init__(self, csv_dir_path,  transforms=None):
        ### numpy array for entire dataset
        self.dataset = pd.read_csv(csv_dir_path, usecols=["labels","directory"]).to_numpy()  
        
        ### labels
        self.labels = self.dataset[:,0]
        #print(self.labels)
        ### images
        self.images = self.dataset[:,1]
        #print(self.images)
        
        ### transformations to apply on images
        self.transforms = transforms
        
    def __getitem__(self, index):
        label = torch.tensor(self.labels[index])
        image_dir = self.images[index]
        #print(self.images)
        image = sk.imread(os.path.join(os.getcwd(),image_dir))
        image = resize(image, (100, 100, 3))
        reshaped_image = image
        if self.transforms:
            reshaped_image = self.transforms(reshaped_image)
        return reshaped_image,label 
    
    def __len__(self):
        return len(self.dataset)
    

In [21]:
BATCH_SIZE = 36

In [22]:
TEST_DATASET = CompundCharacterDataset(csv_dir_path="./CMATERdbTest.csv",transforms=transforms.ToTensor())
TEST_LOADER = DataLoader(dataset=TEST_DATASET, batch_size=BATCH_SIZE, shuffle= False )

In [23]:
TRAIN_DATASET = CompundCharacterDataset(csv_dir_path="./CMATERdbTrain.csv",transforms=transforms.ToTensor())
TRAIN_LOADER = DataLoader(dataset=TRAIN_DATASET, batch_size=BATCH_SIZE, shuffle= False )

In [24]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [25]:
len(TEST_DATASET)

472

In [41]:
cnn_layers =  nn.Sequential(
            # Defining a 2D convolution layer
            # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
            # in_channels (int) – Number of channels in the input image. For B&W it is 1.
            # out_channels (int) – Number of channels produced by the convolution. 4 filters
            # kernel_size (int or tuple) – Size of the convolving kernel (3x3)
            # stride (int or tuple, optional) – Stride of the convolution
            # padding (int or tuple, optional) – Padding of 1 added to both sides of the input
            # example x1 = (n, c=3 , h=100 , w=100 )
            nn.Conv2d(3, 4, kernel_size=3, stride=1, padding=1), #in_channels = 1 is a data dependent hyperparameter. It is 1 because the images are in grayscale
            # x2 = (n, c=12 , h=100 , w=100 )
            nn.BatchNorm2d(4), # Normalize output from the activation function. 
            nn.ReLU(inplace=True), # negative elements to zero
            # x2 = (n, c=12 , h=100 , w=100 )
            nn.MaxPool2d(kernel_size=2, stride=2), #Stride is the number of pixels shifts over the input matrix. When the stride is 1 then we move the filters to 1 pixel at a time. When the stride is 2 then we move the filters to 2 pixels at a time and so on
            # x3 = (n, c=12 , h=49 , w=49 )
            # Defining another 2D convolution layer
            nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            # x3 = (n, c=4 , h=49 , w=49 )
            nn.BatchNorm2d(4), # 4 features
            nn.ReLU(inplace=True), # inplace = True will modify the input directly, without allocating any additional output.
            # x3 = (n, c=48 , h=49 , w=49 )
            nn.MaxPool2d(kernel_size=2, stride=2), # Downsamples the input representation by taking the maximum value
            # x4 = (n, c=48 , h=25 , w=25 )
        )
        # outputSize = floor[(inputSize - filterSize + 2 * padding) / stride] + 1
print(cnn_layers)


linear_layers = nn.Sequential(
            nn.Linear(4 * 25 * 25, 159) 
#159 classes are available for the compound classes dataset. It is a data dependent hyperparameter
)

print(linear_layers)

Sequential(
  (0): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU(inplace=True)
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
Sequential(
  (0): Linear(in_features=2500, out_features=159, bias=True)
)


In [42]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.cnn_layer = cnn_layers
        self.linear_layer = linear_layers
    
    def forward(self, x):
        x = self.cnn_layer(x)
        x = x.view(x.shape[0],-1)
        x = self.linear_layer(x)
        return x


In [43]:
net = Net()

In [44]:
optimizer =  optim.Adam(net.parameters(), lr=0.07) # learning rate 
# defining the loss function
criterion =  nn.CrossEntropyLoss()
# change loss function 
# 1. cross entropy 
# 2. 1 / (1 + distance between winning prob and second best prob)
# add 1 and 2 
# checking if GPU is available
if torch.cuda.is_available():
    print('cuda is available')
    #net = net.cuda()
    #criterion = criterion.cuda()

cuda is available


In [45]:
def accuracy_finder(predictions , labels):
    softmax = torch.nn.Softmax(dim=1) 
    predictions = softmax(predictions)
    values, max_indices = torch.max(predictions, dim=1) 
    accuracy = ( max_indices == labels ).sum()/max_indices.size()[0]
    return accuracy

In [48]:
def train(epochs:int , withCustomLoss = False):
    all_training_losses = []
    all_training_acc = []
    min_loss_at_epoch = -1
    global_min_average_loss = torch.tensor(999999999)
    # min average training loss = infinity 
    for epoch in range(epochs):  # loop over the dataset multiple times
        targets = []
        total_loss_epoch = 0
        total_acc_epoch = 0
        batches_traversed = 0 
        for i, data in enumerate(TRAIN_LOADER, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.float() 
            #if torch.cuda.is_available():
            #    inputs = inputs.cuda()
            #    labels = labels.cuda()
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = net(inputs)
            print(outputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            targets.append(labels)
            total_loss_epoch = total_loss_epoch + loss
            training_accuracy = accuracy_finder(predictions=outputs, labels=labels)
            total_acc_epoch = total_acc_epoch + training_accuracy
            batches_traversed = batches_traversed + 1 
        all_training_losses.append(total_loss_epoch/batches_traversed)
        all_training_acc.append(total_acc_epoch/batches_traversed)
        
        #print("epoch : {}, training loss : {}, training accuracy {}".format(epoch+1,((total_loss_epoch/batches_traversed)),((total_acc_epoch/batches_traversed))))
        #save the model with minimum training loss
        avg_training_loss_per_epoch = total_loss_epoch/batches_traversed
        # if min average training loss > avg_training_loss : min average training loss = avg_training_loss
        # save the model 
        
    print('Finished Training')
    return (all_training_losses, all_training_acc)

In [49]:
training_loss, training_acc, min_loss_at_epoch, min_avg_loss = train(epochs=100)

tensor([[-2.4492, -0.3484, -0.3932,  ..., -2.5155, -2.4861, -2.4941],
        [-2.4492, -0.3484, -0.3932,  ..., -2.5155, -2.4861, -2.4941],
        [-2.4492, -0.3484, -0.3932,  ..., -2.5155, -2.4861, -2.4941],
        ...,
        [-2.4492, -0.3484, -0.3932,  ..., -2.5155, -2.4861, -2.4941],
        [-2.4492, -0.3484, -0.3932,  ..., -2.5155, -2.4861, -2.4941],
        [-2.4492, -0.3484, -0.3932,  ..., -2.5155, -2.4861, -2.4941]],
       grad_fn=<AddmmBackward>)
tensor([[-2.5061, -0.3301, -0.3980,  ..., -2.5725, -2.5432, -2.5511],
        [-2.5061, -0.3301, -0.3980,  ..., -2.5725, -2.5432, -2.5511],
        [-2.5061, -0.3301, -0.3980,  ..., -2.5725, -2.5432, -2.5511],
        ...,
        [-2.5061, -0.3301, -0.3980,  ..., -2.5725, -2.5432, -2.5511],
        [-2.5061, -0.3301, -0.3980,  ..., -2.5725, -2.5432, -2.5511],
        [-2.5061, -0.3301, -0.3980,  ..., -2.5725, -2.5432, -2.5511]],
       grad_fn=<AddmmBackward>)
tensor([[-2.5616, -0.3004, -0.3949,  ..., -2.6281, -2.5988, -2.6067]

tensor([[-3.3054, -0.1242, -0.2145,  ..., -3.3722, -3.3429, -3.3507],
        [-3.3054, -0.1242, -0.2145,  ..., -3.3722, -3.3429, -3.3507],
        [-3.3054, -0.1242, -0.2145,  ..., -3.3722, -3.3429, -3.3507],
        ...,
        [-3.3054, -0.1242, -0.2145,  ..., -3.3722, -3.3429, -3.3507],
        [-3.3054, -0.1242, -0.2145,  ..., -3.3722, -3.3429, -3.3507],
        [-3.3054, -0.1242, -0.2145,  ..., -3.3722, -3.3429, -3.3507]],
       grad_fn=<AddmmBackward>)
tensor([[-3.3389, -0.1252, -0.2143,  ..., -3.4057, -3.3764, -3.3842],
        [-3.3389, -0.1252, -0.2143,  ..., -3.4057, -3.3764, -3.3842],
        [-3.3389, -0.1252, -0.2143,  ..., -3.4057, -3.3764, -3.3842],
        ...,
        [-3.3389, -0.1252, -0.2143,  ..., -3.4057, -3.3764, -3.3842],
        [-3.3389, -0.1252, -0.2143,  ..., -3.4057, -3.3764, -3.3842],
        [-3.3389, -0.1252, -0.2143,  ..., -3.4057, -3.3764, -3.3842]],
       grad_fn=<AddmmBackward>)
tensor([[-3.3716, -0.1269, -0.2148,  ..., -3.4383, -3.4090, -3.4168]

tensor([[-3.7946, -0.2124, -0.2770,  ..., -3.8607, -3.8314, -3.8395],
        [-3.7946, -0.2124, -0.2770,  ..., -3.8607, -3.8314, -3.8395],
        [-3.7946, -0.2124, -0.2770,  ..., -3.8607, -3.8314, -3.8395],
        ...,
        [-3.7946, -0.2124, -0.2770,  ..., -3.8607, -3.8314, -3.8395],
        [-3.7946, -0.2124, -0.2770,  ..., -3.8607, -3.8314, -3.8395],
        [-3.7946, -0.2124, -0.2770,  ..., -3.8607, -3.8314, -3.8395]],
       grad_fn=<AddmmBackward>)
tensor([[-3.8163, -0.2199, -0.2830,  ..., -3.8824, -3.8531, -3.8612],
        [-3.8163, -0.2199, -0.2830,  ..., -3.8824, -3.8531, -3.8612],
        [-3.8163, -0.2199, -0.2830,  ..., -3.8824, -3.8531, -3.8612],
        ...,
        [-3.8163, -0.2199, -0.2830,  ..., -3.8824, -3.8531, -3.8612],
        [-3.8163, -0.2199, -0.2830,  ..., -3.8824, -3.8531, -3.8612],
        [-3.8163, -0.2199, -0.2830,  ..., -3.8824, -3.8531, -3.8612]],
       grad_fn=<AddmmBackward>)
tensor([[-3.8376, -0.2276, -0.2891,  ..., -3.9037, -3.8744, -3.8825]

IndexError: Target 159 is out of bounds.